In [1]:
import requests
import json
import os

# 気象庁データの取得
# jma_url = "https://www.jma.go.jp/bosai/forecast/data/forecast/130000.json"
jma_url = "https://weather.tsukumijima.net/api/forecast/city/130010"
jma_json = requests.get(jma_url).json()

# 取得したいデータを選ぶ
# jma_date = jma_json[0]["timeSeries"][0]["timeDefines"][0]
# jma_weather = jma_json[0]["timeSeries"][0]["areas"][0]["weathers"][0]
# jma_rainfall = jma_json["Feature"][0]["Property"]["WeatherList"]["Weather"][0]["Rainfall"]
# # 全角スペースの削除
# jma_weather = jma_weather.replace('　', '')

# print(jma_date)
# print(jma_weather)
# print(jma_rainfall)

In [2]:
jma_json

{'publicTime': '2025-05-06T17:00:00+09:00',
 'publicTimeFormatted': '2025/05/06 17:00:00',
 'publishingOffice': '気象庁',
 'title': '東京都 東京 の天気',
 'link': 'https://www.jma.go.jp/bosai/forecast/#area_type=offices&area_code=130000',
 'description': {'publicTime': '2025-05-06T16:39:00+09:00',
  'publicTimeFormatted': '2025/05/06 16:39:00',
  'headlineText': '',
  'bodyText': '\u3000東海道沖には低気圧があって、東北東へ進んでいます。また、関東甲信地方は湿った空気の影響を受けています。\n\n\u3000東京地方は、雨となっています。\n\n\u3000６日は、低気圧が関東の東へ進む見込みです。このため、雨で夜は曇りとなるでしょう。伊豆諸島では雨で雷を伴う所がある見込みです。\n\n\u3000７日は、高気圧に覆われますが、寒気や気圧の谷の影響を受ける見込みです。このため、曇り時々晴れで、明け方まで雨となるでしょう。\n\n【関東甲信地方】\n\u3000関東甲信地方は、雨となっています。\n\n\u3000６日は、低気圧が関東の東へ進む見込みです。このため、雨や曇りとなり、雷を伴う所があるでしょう。\n\n\u3000７日は、高気圧に覆われますが、寒気や気圧の谷の影響を受ける見込みです。このため、曇りや晴れで、雨や雷雨となる所があるでしょう。\n\n\u3000関東地方と伊豆諸島の海上では、６日から７日にかけて、うねりを伴いしけるでしょう。また、所々で霧が発生しています。船舶は高波や視程障害に注意してください。',
  'text': '\u3000東海道沖には低気圧があって、東北東へ進んでいます。また、関東甲信地方は湿った空気の影響を受けています。\n\n\u3000東京地方は、雨となっています。\n\n\u3000６日は、低気圧が関東の東へ進む見込みです。このため、雨で夜は曇りとなるでしょう。伊豆

In [3]:
import pandas as pd

weather_df = pd.read_csv("../../data/weather_data.csv", encoding="shift-jis", skiprows=[0,1,2,4,5])

In [4]:

weather_df = weather_df[["年月日", "最高気温(℃)", "最低気温(℃)", "天気概況(昼：06時〜18時)"]]
weather_df = weather_df.rename(columns={
    "年月日": "date",
    "最高気温(℃)": "max_temp",
    "最低気温(℃)": "min_temp",
    "天気概況(昼：06時〜18時)": "weather",
})

In [5]:
weather_df["date"] = pd.to_datetime(weather_df["date"], format="%Y/%m/%d")

In [6]:
import  zipfile
import os

In [7]:
zip_dir = os.path.expanduser('../../data/power_usage')
result = []

In [8]:
for zip_name in sorted(os.listdir(zip_dir)):
    if not zip_name.endswith('.zip'):
        continue
    
    zip_path = os.path.join(zip_dir, zip_name)
    
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        for csv_filename in zip_ref.namelist():
            
            if not csv_filename.endswith('.csv'):
                continue
            
            with zip_ref.open(csv_filename) as csv_file:
                try:
                    df = pd.read_csv(csv_file, encoding="shift-jis", skiprows=54)
                    max_power = df["当日実績(５分間隔値)(万kW)"].max()
                    result.append({
                        "date": csv_filename.split('_')[0],
                        "max_power": max_power
                    })
                except Exception as e:
                    print(f"Error reading {csv_filename}: {e}")
                    

In [9]:
power_usage_df = pd.DataFrame(result)
power_usage_df["date"] = pd.to_datetime(power_usage_df["date"], format="%Y%m%d")

In [10]:
integrated_df = pd.merge(weather_df, power_usage_df, on="date", how="inner")

In [11]:
integrated_df

,date,max_temp,min_temp,weather,max_power
0,2022-04-01,12.4,3.9,曇一時雨,3877
1,2022-04-02,12.9,3.6,薄曇時々晴,3351
2,2022-04-03,10.1,6.6,雨一時曇,3546
3,2022-04-04,9.8,6.7,大雨,4185
4,2022-04-05,17.9,8.2,曇一時雨後晴,3663
...,...,...,...,...,...
1001,2024-12-27,12.7,3.8,晴後一時曇,3789
1002,2024-12-28,11.5,1.4,晴後一時曇,3622
1003,2024-12-29,12.3,0.5,快晴,3476
1004,2024-12-30,10.3,3.5,薄曇一時晴,3466


In [12]:
integrated_df["weather"].unique()

array(['曇一時雨', '薄曇時々晴', '雨一時曇', '大雨', '曇一時雨後晴', '曇後晴', '曇', '快晴', '薄曇',
       '雨', '曇時々晴一時雨', '曇時々雨', '曇時々晴', '晴時々曇', '曇後雨', '晴', '曇一時晴',
       '曇時々雨後一時晴', '薄曇一時晴', '薄曇後一時晴', '雨後薄曇', '雨時々曇', '晴後薄曇', '曇時々大雨',
       '晴一時曇', '大雨後曇', '晴後一時曇', '薄曇後時々雨、雷を伴う', '晴一時薄曇', '曇時々雨後晴', '曇後時々雨',
       '雨時々曇一時霧雨', '曇後一時晴', '薄曇後一時雨', '晴時々薄曇', '薄曇後晴', '曇後時々晴', '大雨時々曇',
       '雨時々霧雨一時曇', '曇時々晴一時雨、雷を伴う', '曇時々雨、雷を伴う', '曇時々雨一時晴', '雨一時曇、雷を伴う',
       '雨時々曇一時晴', '雨後晴一時曇', '曇後一時雨', '曇時々雨一時晴、雷を伴う', '大雨時々曇、雷を伴う', '晴後曇',
       '雨後一時曇', '晴後時々薄曇', '晴後時々曇', '薄曇時々雨一時晴', '晴時々曇一時雨', '雨後一時晴',
       '雨時々曇後晴', '雨後曇', '晴一時雨', '雨後晴時々曇', '薄曇時々雨後一時晴', '晴後一時薄曇',
       '雪一時みぞれ後雨', '晴後一時雨', '雨後時々曇', '雨後曇時々晴', '晴後雨時々曇',
       '曇時々晴一時雨、雷・ひょうを伴う', '晴後雨時々曇、雷を伴う', '曇一時雨後時々晴', '曇一時雨後一時晴',
       '薄曇時々晴一時雨', '曇一時晴後一時雨', '曇後時々雨、雷を伴う', '晴後時々曇一時雨', '晴時々雨一時曇、雷を伴う',
       '曇一時晴後雨、雷を伴う', '曇後一時雨、雷を伴う', '霧雨後曇一時雨', '大雨後時々曇一時晴', '曇後時々晴一時雨',
       '晴時々雨一時曇', '雨後曇一時晴', '晴時々薄曇一時雨', '晴時々曇一時雪、雷を伴う', '雨後時々曇一時晴',
       '曇後雪時々雨一時みぞれ', '薄曇後時々

In [13]:
def weather_check(weather: str) -> str:
    """天気の文字列を基本的なカテゴリに分類する関数
    
    Args:
        weather: 元の天気の説明文字列
        
    Returns:
        str: 分類された天気カテゴリ
            快晴、晴れ、晴れ時々曇り、晴れ時々雨、曇り、曇り時々雨、雨、
            雷雨、晴れ（雷あり）、曇り（雷あり）、雷、霧・もや、その他、不明 (NaN値の場合)
    """
    if pd.isna(weather):
        return "不明"
    
    # 雪系
    elif any(keyword in weather for keyword in ["雪", "ゆき"]):
        return "雪"
    
    # 雷系
    if "雷" in weather:
        if any(keyword in weather for keyword in ["雨", "あめ"]):
            return "雷雨"
        elif any(keyword in weather for keyword in ["晴", "日射"]):
            return "晴れ(雷あり)"
        elif any(keyword in weather for keyword in ["曇", "くもり"]):
            return "曇り(雷あり)"
        else:
            return "雷"
    
    # 晴れ系
    if "快晴" in weather:
        return "快晴"
    elif any(keyword in weather for keyword in ["晴", "日射"]):
        if any(keyword in weather for keyword in ["曇", "くもり"]):
            return "晴れ時々曇り"
        elif any(keyword in weather for keyword in ["雨", "あめ", "雷"]):
            return "晴れ時々雨"
        else:
            return "晴れ"
    
    # 曇り系
    elif any(keyword in weather for keyword in ["曇", "くもり"]):
        if any(keyword in weather for keyword in ["雨", "あめ"]):
            return "曇り時々雨"
        else:
            return "曇り"
    
    # 雨系
    elif any(keyword in weather for keyword in ["雨", "あめ"]):
        return "雨"
    
    # その他
    else:
        return "その他"

integrated_df["weather_category"] = integrated_df["weather"].apply(weather_check)

In [14]:
integrated_df["weather_category"].value_counts()

weather_category
晴れ時々曇り     407
曇り時々雨      175
快晴         126
曇り         120
晴れ         112
雨           29
雷雨          22
晴れ時々雨        8
雪            5
晴れ(雷あり)      2
Name: count, dtype: int64

In [15]:
integrated_df

,date,max_temp,min_temp,weather,max_power,weather_category
0,2022-04-01,12.4,3.9,曇一時雨,3877,曇り時々雨
1,2022-04-02,12.9,3.6,薄曇時々晴,3351,晴れ時々曇り
2,2022-04-03,10.1,6.6,雨一時曇,3546,曇り時々雨
3,2022-04-04,9.8,6.7,大雨,4185,雨
4,2022-04-05,17.9,8.2,曇一時雨後晴,3663,晴れ時々曇り
...,...,...,...,...,...,...
1001,2024-12-27,12.7,3.8,晴後一時曇,3789,晴れ時々曇り
1002,2024-12-28,11.5,1.4,晴後一時曇,3622,晴れ時々曇り
1003,2024-12-29,12.3,0.5,快晴,3476,快晴
1004,2024-12-30,10.3,3.5,薄曇一時晴,3466,晴れ時々曇り


In [16]:
import datetime
import holidays
import numpy as np

JP_HOLIDAY = holidays.Japan()

In [17]:
def make_features(df: pd.DataFrame) -> pd.DataFrame:
    """データフレーム全体に対して特徴量を作成する関数
    
    Args:
        df: pd.DataFrame（date, max_temp, min_temp, weather_category列を含む）
        
    Returns:
        pd.DataFrame: 特徴量を追加したデータフレーム
    """
    result_df = df.copy()
    
    # 数値系特徴量
    result_df["avg"] = (df["max_temp"] + df["min_temp"]) / 2
    result_df["rng"] = df["max_temp"] - df["min_temp"]
    result_df["cdd"] = (result_df["avg"] - 18).clip(lower=0)
    result_df["hdd"] = (18 - result_df["avg"]).clip(lower=0)
    result_df["hot"] = (df["max_temp"] >= 30).astype(int)
    result_df["cold"] = (df["min_temp"] <= 5).astype(int)
    
    # カレンダー系特徴量
    result_df["dow"] = df["date"].dt.weekday
    result_df["month"] = df["date"].dt.month
    result_df["dow_sin"] = np.sin(2 * np.pi * result_df["dow"] / 7)
    result_df["dow_cos"] = np.cos(2 * np.pi * result_df["dow"] / 7)
    result_df["mon_sin"] = np.sin(2 * np.pi * result_df["month"] / 12)
    result_df["mon_cos"] = np.cos(2 * np.pi * result_df["month"] / 12)
    # 休日フラグ
    result_df["weekend"] = (result_df["dow"] >= 5).astype(int)
    # 祝日フラグ
    result_df["holiday"] = result_df["date"].apply(lambda x: int(x in JP_HOLIDAY))
    
    return result_df

In [25]:
from common.feature_encoder import FeatureEncoder
from omegaconf import OmegaConf, DictConfig
from typing import Dict

In [19]:
df = make_features(integrated_df)

In [ ]:
encoders_dict = {}


In [ ]:
# configファイルを読み込み、天気カテゴリーにOne-Hotエンコーディングを適用する
#
# config.yamlには以下のような設定が記述されています：
# encoders:
#   - name: weather_encoder    # エンコーダーの名前
#     type: one-hot           # エンコーディングの種類（one-hotまたはordinal）
#     columns:                # エンコード対象のカラム
#       - weather_category    # カラム名
#     drop_first: False       # 最初のカテゴリーを除外するかどうか
#     prefix: weather         # 生成される列の接頭辞
#     prefix_separator: _     # 接頭辞とカテゴリー値の区切り文字
#     sparse: False           # スパース行列として出力するかどうか
#
# 使用例：
# config_path = 'config.yaml'
# config = OmegaConf.load(config_path)
# encoded_df, encoders_dict = encode_features(df, config, encoders_dict)

# 設定ファイルを読み込み
config_path = 'config.yaml'
config = OmegaConf.load(config_path)

# カラムリストを表示
print("One-Hotエンコーダー対象カラム:")
print(config["encoders"][0]["columns"])

# 特徴量をエンコード
encoded_df, encoders_dict = encode_features(df, config, encoders_dict)

In [26]:
def encode_features(df: pd.DataFrame, config: DictConfig, encoders_dict: Dict[str, FeatureEncoder]):
    """特徴量をエンコードする関数
    
    Args: Dictconfig
        config: 設定ファイルの内容
        encoders_dict: エンコーダー辞書
        df: pd.DataFrame（特徴量を含む）
        
    Returns:
        pd.DataFrame: エンコードされたデータフレーム
    """
    if "encoder" in config:
        for params in config["encoders"]:
            if params["name"] not in encoders_dict:
                encoder = FeatureEncoder(**params)
                df = encoder.fit_transform(df)
                encoders_dict[params["name"]] = encoder
            else:
                encoder = encoders_dict[params["name"]]
                df = encoder.transform(df)
    return df, encoders_dict


NameError: name 'Dictconfig' is not defined

In [20]:
encoder = FeatureEncoder()

,date,max_temp,min_temp,weather,max_power,weather_category,avg,rng,cdd,hdd,hot,cold,dow,month,dow_sin,dow_cos,mon_sin,mon_cos,weekend,holiday
0,2022-04-01,12.4,3.9,曇一時雨,3877,曇り時々雨,8.15,8.5,0.0,9.85,0,1,4,4,-0.433884,-0.900969,8.660254e-01,-0.5,0,0
1,2022-04-02,12.9,3.6,薄曇時々晴,3351,晴れ時々曇り,8.25,9.3,0.0,9.75,0,1,5,4,-0.974928,-0.222521,8.660254e-01,-0.5,1,0
2,2022-04-03,10.1,6.6,雨一時曇,3546,曇り時々雨,8.35,3.5,0.0,9.65,0,0,6,4,-0.781831,0.623490,8.660254e-01,-0.5,1,0
3,2022-04-04,9.8,6.7,大雨,4185,雨,8.25,3.1,0.0,9.75,0,0,0,4,0.000000,1.000000,8.660254e-01,-0.5,0,0
4,2022-04-05,17.9,8.2,曇一時雨後晴,3663,晴れ時々曇り,13.05,9.7,0.0,4.95,0,0,1,4,0.781831,0.623490,8.660254e-01,-0.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,2024-12-27,12.7,3.8,晴後一時曇,3789,晴れ時々曇り,8.25,8.9,0.0,9.75,0,1,4,12,-0.433884,-0.900969,-2.449294e-16,1.0,0,0
1002,2024-12-28,11.5,1.4,晴後一時曇,3622,晴れ時々曇り,6.45,10.1,0.0,11.55,0,1,5,12,-0.974928,-0.222521,-2.449294e-16,1.0,1,0
1003,2024-12-29,12.3,0.5,快晴,3476,快晴,6.40,11.8,0.0,11.60,0,1,6,12,-0.781831,0.623490,-2.449294e-16,1.0,1,0
1004,2024-12-30,10.3,3.5,薄曇一時晴,3466,晴れ時々曇り,6.90,6.8,0.0,11.10,0,1,0,12,0.000000,1.000000,-2.449294e-16,1.0,0,0
